## Financial data storage

### Storing data on disk in CSV format

In [1]:
import pandas as pd
from openbb import obb
from IPython.display import Markdown, display
obb.user.preferences.output_type = "dataframe"

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",)
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

In [4]:
def save_data_range(symbol, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_csv(f"{symbol}.gz", compression="gzip", index=False)

In [5]:
def get_data(symbol):
    return pd.read_csv(f"{symbol}.gz",
                       compression="gzip",
                       index_col="date",
                       usecols=["date", "open", "high", "low", "close", "volume", "symbol"],)

In [6]:
save_data_range("AAPL")

In [7]:
apple = get_data("AAPL")
apple

,open,high,low,close,volume,symbol
date,,,,,,
2024-01-18,186.089996,189.139999,185.830002,188.630005,78005800,AAPL
2024-01-19,189.330002,191.949997,188.820007,191.559998,68741000,AAPL
2024-01-22,192.300003,195.330002,192.259995,193.889999,60133900,AAPL
2024-01-23,195.020004,195.750000,193.830002,195.179993,42355600,AAPL
2024-01-24,195.419998,196.380005,194.339996,194.500000,53631300,AAPL
...,...,...,...,...,...,...
2025-01-13,233.529999,234.669998,229.720001,234.399994,49630700,AAPL
2025-01-14,234.750000,236.119995,232.470001,233.279999,39435300,AAPL
2025-01-15,234.639999,238.960007,234.429993,237.869995,39832000,AAPL


In [8]:
apple.to_csv("market_data.csv")

In [9]:
apple.to_csv("market_data.tsv", sep="\t")

### Storing data on disk with SQLite

In [10]:
from sys import argv
import sqlite3
import pandas as pd
import exchange_calendars as xcals
from openbb import obb
obb.user.preferences.output_type = "dataframe"

In [11]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",)
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

In [12]:
def save_data_range(symbol, conn, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", conn, if_exists="replace", index=False)

In [13]:
def save_last_trading_session(symbol, conn, today):
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", conn, if_exists="append", index=False)

In [14]:
if __name__ == "__main__":
    conn = sqlite3.connect("market_data.sqlite")
    if argv[1] == "bulk":
        symbol = argv[2]
        start_date = argv[3]
        end_date = argv[4]
        save_data_range(symbol, conn, start_date=None, end_date=None)
        print(f"{symbol} saved between {start_date} and {end_date}")
    elif argv[1] == "last":
        symbol = argv[2]
        calendar = argv[3]
        cal = xcals.get_calendar(calendar)
        today = pd.Timestamp.today().date()
        if cal.is_session(today):
            save_last_trading_session(symbol, conn, today)
            print(f"{symbol} saved")
        else:
            print(f"{today} is not a trading day. Do nothing.")
    else:
        print("Enter bulk or last")        

Enter bulk or last


In [15]:
xcals.get_calendar_names(include_aliases=False)

['24/5',
 '24/7',
 'AIXK',
 'ASEX',
 'BVMF',
 'CMES',
 'IEPA',
 'XAMS',
 'XASX',
 'XBKK',
 'XBOG',
 'XBOM',
 'XBRU',
 'XBSE',
 'XBUD',
 'XBUE',
 'XCBF',
 'XCSE',
 'XDUB',
 'XDUS',
 'XEEE',
 'XETR',
 'XFRA',
 'XHAM',
 'XHEL',
 'XHKG',
 'XICE',
 'XIDX',
 'XIST',
 'XJSE',
 'XKAR',
 'XKLS',
 'XKRX',
 'XLIM',
 'XLIS',
 'XLON',
 'XLUX',
 'XMAD',
 'XMEX',
 'XMIL',
 'XMOS',
 'XNYS',
 'XNZE',
 'XOSL',
 'XPAR',
 'XPHS',
 'XPRA',
 'XSAU',
 'XSES',
 'XSGO',
 'XSHG',
 'XSTO',
 'XSWX',
 'XTAE',
 'XTAI',
 'XTKS',
 'XTSE',
 'XWAR',
 'XWBO',
 'us_futures']

### Storing data in HDF5 format

In [16]:
import pandas as pd
from openbb import obb
obb.user.preferences.output_type = "dataframe"
STOCKS_DATA_STORE = "stocks.h5"
FUTURES_DATA_STORE = "futures.h5"
ticker = "SPY"
root = "CL"

In [17]:
spy_equity = obb.equity.price.historical(ticker,
                                         start_date="2021-01-01",
                                         provider="yfinance")
spy_chains = obb.derivatives.options.chains(ticker,
                                            provider="cboe")
spy_expirations = (spy_chains.expiration.astype(str).unique().tolist())
spy_historic = (obb.equity.price.historical(ticker + spy_expirations[-10].replace("-", "")[2:] + "C" + "00400000",
                                            start_date="2021-01-01",
                                            provider="yfinance"))

In [18]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    store.put("equities/spy/stock_prices", spy_equity)
    store.put("equities/spy/options_prices", spy_historic)
    store.put("equities/spy/chains", spy_chains)

In [19]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    spy_prices = store["equities/spy/stock_prices"]
    spy_options = store["equities/spy/options_prices"]
    spy_chains = store["equities/spy/chains"]

In [20]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    for i in range(24, 31):
        expiry = f"20{i}-12"
        df = obb.derivatives.futures.historical(symbol=[root], expiry=expiry, start_date="2021-01-01",)
        df.rename(columns={"close": expiry}, inplace=True)
        prices = df[expiry]
        store.put(f"futures/{root}/{expiry}", prices)

In [21]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    es_prices = store[f"futures/{root}/2024-12"]

In [22]:
display(spy_prices)
display(spy_options)
display(es_prices)

,open,high,low,close,volume,dividend
date,,,,,,
2021-01-04,375.309998,375.450012,364.820007,368.790009,110210800,0.0
2021-01-05,368.100006,372.500000,368.049988,371.329987,66426200,0.0
2021-01-06,369.709991,376.980011,369.119995,373.549988,107997700,0.0
2021-01-07,376.100006,379.899994,375.910004,379.100006,68766800,0.0
2021-01-08,380.589996,381.489990,377.100006,381.260010,71677200,0.0
...,...,...,...,...,...,...
2025-01-13,575.770020,581.750000,575.349976,581.390015,47910100,0.0
2025-01-14,584.359985,585.000000,578.349976,582.190002,48420600,0.0
2025-01-15,590.330017,593.940002,589.200012,592.780029,56900200,0.0


,open,high,low,close,volume
date,,,,,
2024-04-04,147.000000,147.000000,147.000000,147.000000,1
2024-05-02,131.429993,131.429993,131.429993,131.429993,4
2024-05-20,156.570007,156.570007,156.320007,156.320007,6
2024-06-03,150.940002,150.940002,150.940002,150.940002,0
2024-06-10,159.199997,159.199997,159.199997,159.199997,1
2024-06-12,168.470001,168.470001,168.470001,168.470001,0
2024-06-17,170.660004,170.660004,170.660004,170.660004,1
2024-06-27,170.169998,170.229996,170.169998,170.229996,3
2024-07-08,178.020004,178.020004,178.020004,178.020004,1


date
2021-01-04    47.619999
2021-01-05    49.930000
2021-01-06    50.630001
2021-01-07    50.830002
2021-01-08    52.240002
                ...    
2025-01-10    76.570000
2025-01-13    78.820000
2025-01-14    77.500000
2025-01-15    80.040001
2025-01-17    78.040001
Name: 2024-12, Length: 1016, dtype: float64